In [ ]:
#!unzip /content/speech-emotion-recognition-ravdess-data.zip


In [6]:
# OS BASED PATH
# base = Path(__file__).resolve().parent
# Notebook Path
import os
import glob

# All files and directories ending with .txt and that don't begin with a dot:
ROOT = os.path.abspath('')
dataset_path = os.path.join(ROOT, 'RAVDESS_dataset')

dataset_files = glob.glob(f"{dataset_path}/Actor_*/*.wav")

In [7]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [12]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [8]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [9]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
focused_emotions=['calm', 'happy', 'fearful', 'disgust', 'angry']

In [10]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in dataset_files:
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in focused_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [13]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [14]:
x_train

array([[-5.82729675e+02,  4.92052765e+01,  1.56372023e+01, ...,
         2.17359353e-04,  1.11733614e-04,  3.87394139e-05],
       [-6.90802979e+02,  5.52862778e+01,  9.39893723e-03, ...,
         1.56554834e-05,  6.50626635e-06,  4.85508917e-06],
       [-5.15834351e+02,  5.33766747e+01, -4.80185843e+00, ...,
         5.82872541e-04,  2.96204584e-04,  2.06498327e-04],
       ...,
       [-6.91606018e+02,  6.10901642e+01,  1.26523800e+01, ...,
         8.28699285e-06,  4.08341111e-06,  1.50944322e-06],
       [-4.84353149e+02,  4.05667686e+01, -1.36543274e+01, ...,
         3.19829705e-04,  2.68833537e-04,  1.59812189e-04],
       [-6.77559570e+02,  4.86538773e+01,  4.00112343e+00, ...,
         3.61699349e-05,  2.35095704e-05,  1.31121878e-05]])

In [15]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(720, 240)


In [16]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [17]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [18]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [19]:
#Predict for the test set
y_pred=model.predict(x_test)

In [20]:
y_pred

array(['calm', 'angry', 'disgust', 'calm', 'fearful', 'fearful',
       'fearful', 'angry', 'happy', 'happy', 'disgust', 'fearful',
       'angry', 'angry', 'happy', 'calm', 'happy', 'calm', 'happy',
       'happy', 'disgust', 'disgust', 'happy', 'calm', 'fearful',
       'disgust', 'disgust', 'angry', 'fearful', 'angry', 'happy',
       'disgust', 'calm', 'calm', 'calm', 'angry', 'disgust', 'disgust',
       'angry', 'angry', 'happy', 'happy', 'calm', 'disgust', 'disgust',
       'angry', 'fearful', 'disgust', 'happy', 'fearful', 'angry', 'calm',
       'disgust', 'calm', 'calm', 'calm', 'disgust', 'angry', 'angry',
       'happy', 'disgust', 'angry', 'angry', 'calm', 'disgust', 'fearful',
       'angry', 'disgust', 'happy', 'fearful', 'disgust', 'fearful',
       'fearful', 'disgust', 'angry', 'calm', 'disgust', 'disgust',
       'calm', 'fearful', 'happy', 'calm', 'fearful', 'disgust',
       'disgust', 'angry', 'happy', 'angry', 'fearful', 'disgust',
       'angry', 'calm', 'calm',

In [21]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.83%


In [22]:
from sklearn.metrics import accuracy_score, f1_score

In [23]:
f1_score(y_test, y_pred,average=None)

array([0.77777778, 0.8       , 0.6728972 , 0.61176471, 0.65882353])

In [24]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,calm
1,angry,angry
2,disgust,disgust
3,calm,calm
4,fearful,fearful
5,happy,fearful
6,disgust,fearful
7,angry,angry
8,disgust,happy
9,happy,happy


In [25]:
# import pickle
# # Writing different model files to file
# with open( 'modelForPrediction1.sav', 'wb') as f:
#     pickle.dump(model,f)
import joblib

clf_file = 'emotion_classifier_v2.pkl'

joblib.dump(model, clf_file)

['emotion_classifier_v2.pkl']

In [27]:
filename = 'emotion_classifier_v2.pkl'
loaded_model = joblib.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(f"{dataset_path}/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [28]:
feature

array([[-6.93497009e+02,  5.00643883e+01,  5.71450472e-01,
         1.43299656e+01,  3.33637023e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119989e+01, -7.29413891e+00,
         1.74018884e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789333e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803221e+00, -5.68717480e-01,
        -6.47753334e+00, -1.24320579e+00, -2.80542541e+00,
        -5.43635845e+00, -4.46875453e-01, -3.63516617e+00,
        -2.98372650e+00, -5.63902617e-01, -1.65101981e+00,
        -5.55944800e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821268e+00,
        -5.54154396e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577050e+00, -1.62457860e+00, -2.03990722e+00,
        -3.62910819e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345674e-01,
         6.60464048e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921480e-01,  7.23420441e-01,  7.16119647e-0